In [91]:
import pandas as pd
import datetime
import time
import math

In [84]:
folder = '12_11_2018_19_16'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [85]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [96]:
res_eval['diff_p'] = (res_eval['pred_p'] - res_eval['real_p'])
res_eval['diff_p'] = res_eval['diff_p']*res_eval['diff_p']

In [101]:
res_eval.groupby('type')[['diff_p']].mean()

,diff_p
type,
CNN,0.090749
CRNN,0.016511
DNN,0.033400
FeatureBased_Bayesian Ridge Regression,0.070919
FeatureBased_Lasso Regression,0.070919
FeatureBased_Linear Regression,0.070919
FeatureBased_Passive Aggressive Regression,0.307341
FeatureBased_SGD Regression,0.448331
FeatureBased_SVM,0.070919


In [44]:
res_eval.groupby('type')[['real_p','pred_p']].corr().iloc[0::2,-1]

type                                              
CNN                                         real_p   -0.033945
CRNN                                        real_p    0.776831
DNN                                         real_p    0.166460
FeatureBased_Bayesian Ridge Regression      real_p   -0.009344
FeatureBased_Lasso Regression               real_p   -0.009344
FeatureBased_Linear Regression              real_p   -0.009344
FeatureBased_Passive Aggressive Regression  real_p   -0.008453
FeatureBased_SGD Regression                 real_p   -0.019098
FeatureBased_SVM                            real_p   -0.009344
FeatureBased_Theil-Sen Regression           real_p   -0.011250
GRU                                         real_p    0.199635
Name: pred_p, dtype: float64

In [88]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [90]:
res_adapt.groupby('type').max()

,Unnamed: 0,instance,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,,
CNN,84969,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,0.499668,0.5,0.499834,0.499335,0.300000,0.499169
CRNN,84971,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,1.000000,1.0,1.000000,0.993767,0.315391,0.743541
DNN,84970,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,0.499668,0.5,0.499834,0.499335,0.300000,0.499169
GRU,84968,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,0.499668,0.5,0.499834,0.499335,0.300000,0.499169
IR,84966,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,1.000000,1.0,1.000000,0.535157,0.611542,0.609934
SVDpp,84967,99 WordNet Jiang Conrath Ontobuilder Union Dom...,0,1.0,1.0,1.0,1.000000,1.0,1.000000,0.659752,0.388313,0.646117


In [86]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['pred_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['pred_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [87]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
CNN_CNN,0.494517,0.500000,0.497238
CNN_CRNN,0.494517,0.500000,0.497238
CNN_DNN,0.494517,0.500000,0.497238
CNN_GRU,0.494517,0.500000,0.497238
CRNN_CNN,0.648268,0.597975,0.604287
CRNN_CRNN,0.648230,0.597959,0.604259
CRNN_GRU,0.648249,0.597989,0.604283
DNN_CNN,0.494517,0.500000,0.497238
DNN_CRNN,0.527832,0.509754,0.510091


In [21]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

ValueError: Must pass DataFrame with boolean values only